# Inspecting and Monitoring Deep-Learning Models Using Keras Callbacks and TensorBoard
Until now there was very little contron of running deep learning models after using `model.fit()` or `model.fit_generator`.

## Using callbacks to act on a model during training
When training a model it is difficult to predict how many epochs will be needed to obtain an optimal validation or loss. A good way to determine when to stop training is when it is measured that the validation loss is no longer improving. A *callback* is an object that is passed to the model in the call to *fit* and that is called by the model at various points during training. A callback object has access to all the available data about the state of the model and its performance, and it can take action:
    * interrupt training
    * save a model
    * load a different weight set
    * alter the state of he model
    
Some examples of using callbacks are:
    * Model checkpointing-saving the current weights of the model at different points during training (keras.callbacks.ModelCheckpoint)
    * Early stopping-interrupting training when the validation loss is no longer imrpoving (keras.callbacks.EarlyStopping)
    * Dynamically adjusting the value of certain parameters during training-such as adjusting the learning rate of the optimizer (keras.callbacks.LearningRateScheduler, (keras.callbacks.ReduceLROnPlateu, keras.callbacks.CSVLogger)
    * Logging training and validation metrics during training or visualizing the representations learned by the model as they're updated-For example, the keras progress bar

## ModelCheckpoint and EarlyStopping Callbacks
The Early stopping callback can be used to interrupt training once a target metric being monitored has stopped improving for a fixed number of epochs. This allows one to interrupt training as soon as overfitting occurs, this aviding the need to retrain the model for a smaller number of epochs. This is usually used with `ModelCheckpoint` which lets you continiously save the mode during training.

```python
import keras
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='acc',
        patience = 1,
),
    keras.callbacks.ModelCheckpoint(
    filepath = 'my_model.h5',
    monitor = 'val_loss',
    save_best_only = True,
    )
]

model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy'.
              metrics = ['acc'])
model.fit(x, y,
          epochs = 10,
          batch_size = 32,
          callbacks = callbacks_list,
          validation_data = (x_val, y_val))
```

## ReduceLROnPlateau
This callback can be used to reduce the learning rate when validation loss has stopped improving. Reducing or increasing the learning rate in case of a *loss plateau* is an effective strategy to escape a local minima during training.

```python
callbacks_list = [keras.callbacks.ReduceLROnPlateau(minitor = 'val_loss',
                                                    factor = 0.1,
                                                    patience = 10)]
model.fit(x, y,
          epochs = 10,
          batch_size = 32,
          callbacks = callbacks_list,
          validation_data = (x_val, y_val))
```

## Custom Callback
Callbacks are implemented by subclassing the class `keras.callbacks.Callback`. They can be implemented at any number of the following transparantly named mathods, which are called at vairous parts of the training:
```python
# Start|End of every epoch
on_epoch_begin
on_epoch_ebd

# Start|End of every batch
on_batch_begin
on_batch_end

# Start|End of training
on_train_begin
on_train_end
```
These methods are called with a `logs` argument, which essentially is a dictionary containing information about the previous batch, epoch, or training run: training and validation metrics, and so on.

The callback object also has acess to the attributes:
    * `self.model` the model instance from which the callback is being called
    * `self.validation_data` the value of what was passed to `fit` as validation data
    
Here is an example of a custom callback that saves to disk, Numpy arrays, the activation of every layer of the model at the end of every epoch, computed on the first sample of the validation set:

```python
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    
    # Called by the parent model before training, to inform the callback object what model is calling it
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        # Model isntance that returns the activations of all the layers
        self.activation_model = keras.models.Model(model.input,
                                                  layer_outputs)


    def on_epoch_end(self, epoch, logs = None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data')
           
        # gets the first input sample of the validation data
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        
        f = open('Activation_at_epoch' + str(epoch)+'.npz', 'w')
        np.savez(f, activations)
        f.close()
```